In [11]:
!pip install beautifulsoup4


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install requests

  Obtaining dependency information for requests from https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl.metadata
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/57/ec/80c8d48ac8b1741d5b963797b7c0c869335619e13d4744ca2f67fc11c6fc/charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl (99 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
from bs4 import BeautifulSoup
import requests

def products_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Targeting the <ul> element with id "product-grid"
    product_grid = soup.find('ul', id='product-grid')

    # Targeting the <li> elements within the <ul> element
    li_elements = product_grid.find_all('li')

    # Extracting the links from the <li> elements
    all_product = [li.find('a')['href'] for li in li_elements if li.find('a')]

    return all_product

# Example usage:
url = 'https://watchcollectors.co.uk/collections/all'
all_links = products_links(url)
print(len (all_links))
print(all_links)


12
['/products/choaprd', '/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', '/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', '/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', '/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', '/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', '/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', '/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', '/products/cartier-tank-americaine-chronograph-ref-2892-automatic-31mm-18k-yellow-gold', '/products/cartier-tank-americaine-chronograph-ref-3072-automatic-31mm-18k-rose-gold', '/products/cartier-tank-americaine-ref-2927-automatic-31mm-18k-rose-gold', '/products/cartier-tank-americaine-chronograph-ref-3073-a

In [23]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

def products_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Targeting the <ul> element with id "product-grid"
    product_grid = soup.find('ul', id='product-grid')

    # Targeting the <li> elements within the <ul> element
    li_elements = product_grid.find_all('li')

    # Extracting the links from the <li> elements and joining with the base URL
    base_url = 'https://watchcollectors.co.uk'  # Replace with the actual base URL
    all_product = [urljoin(base_url, li.find('a')['href']) for li in li_elements if li.find('a')]

    return all_product

['https://watchcollectors.co.uk/products/choaprd', 'https://watchcollectors.co.uk/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', 'https://watchcollectors.co.uk/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', 'https://watchcollectors.co.uk/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', 'https://watchcollectors.co.uk/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', 'https://watchcollectors.co.uk/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', 'https://watchcollectors.co.uk/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', 'https://watchcollectors.co.uk/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', 'https://watchcollectors.co.uk/products/cartier-tank-americaine-chronograph-ref

In [24]:
# Example usage:
# url = 'https://watchcollectors.co.uk/collections/all'
url = input("Enter Url: ")
all_links = products_links(url)

# Print the list of links
print(all_links)


['https://watchcollectors.co.uk/products/choaprd', 'https://watchcollectors.co.uk/products/louis-vuitton-ref-tambour-xl-minute-repeater-travel-time-18k-rose-gold-box-papers', 'https://watchcollectors.co.uk/products/rolex-lady-datejust-pearlmaster-ref-69298-white-mother-of-pearl-diamond-dial-18k-yellow-gold', 'https://watchcollectors.co.uk/products/rolex-yacht-master-40-ref-126622-slate-dial-box-papers-2021-stainless-steel-platinum', 'https://watchcollectors.co.uk/products/audemars-piguet-royal-oak-chronograph-ref-26603st-oo-d002cr-01000-blue-dial-box-papers-2010-stainless-steel', 'https://watchcollectors.co.uk/products/rolex-daytona-ref-116520-black-dial-2012-box-papers-stainless-steel-1', 'https://watchcollectors.co.uk/products/rolex-daytona-beach-blue-ref-116509-18k-white-gold-2007', 'https://watchcollectors.co.uk/products/vacheron-constantin-malte-chronograph-ref-49180-champagne-dial-40mm-18k-rose-gold', 'https://watchcollectors.co.uk/products/cartier-tank-americaine-chronograph-ref